# Bike Montreal data science (Daily user predicton)

**Diclaimer**: If anyone is reading this, please note that this project is my first data science project. It is my first time using Jupyter notebooks, pandas, SQL and many other things. I have also deliberately decided to not follow a similar project as an example because I feared I would simply copy the work and not learn as much in consequence.  This means that this project will **not** follow best practices in the industry or even correct usage of tools and libraries. My goal was to blindly jump into a project and learn as I go.

## Objective

[Bixi](https://bixi.com/en) is Montreal's public bike share program. It has been in service since 2014 and now has close to 800 stations with close to 10000 bikes in the network. 

In a previous project where I first learned Python and webscrapping, I scrapped Bixi's open data since 2014 and the historical daily weather of Montreal. 

In this project, I will attempt to predict the daily usage of Bixi bikes based on the different trends and weather condition.

## Sections (TODO)

In [109]:
import helpers
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

BIXI_TABLE_ID = 'bixi-montreal.bixi.all-rides'
WEATHER_TABLE_ID = 'bixi-montreal.bixi.mtl-weather'



## Creating our dataframe

Let's query from **gbq** the first and last day of bixi and weather data and see if there is any weather data missing.

In [110]:
first_day_bixi = helpers.get_min_value(BIXI_TABLE_ID, 'start_date')
last_day_bixi = helpers.get_max_value(BIXI_TABLE_ID, 'start_date')
first_day_weather = helpers.get_min_value(WEATHER_TABLE_ID, 'DATE')
last_day_weather = helpers.get_max_value(WEATHER_TABLE_ID, 'DATE')

enough_min = first_day_weather <= first_day_bixi
enough_max = last_day_weather >= last_day_bixi

if enough_min != True and enough_max != True:
  print(f'Need weather data since {first_day_bixi} and until {last_day_bixi}')
elif enough_min != True:
  print(f'Need weather data since {first_day_bixi}')
elif enough_max != True:
  print(f'Need weather data until {last_day_bixi}')
else:
  print('You have all the weather data you needed!')

You have all the weather data you needed!


Create our main dataframe called main_df

In [111]:
# create empty dataframe
main_df = pd.DataFrame(columns=('DATE', 'DAILY_COUNT', 'MEAN_TEMP', 'TOTAL_RAIN', 'TOTAL_SNOW'))


In [112]:
first_day_bixi = helpers.get_min_value(BIXI_TABLE_ID, 'start_date')
last_day_bixi = helpers.get_max_value(BIXI_TABLE_ID, 'start_date')

start_date = datetime.date(first_day_bixi.year, first_day_bixi.month, first_day_bixi.day)
end_date = datetime.date(last_day_bixi.year, last_day_bixi.month, last_day_bixi.day)

daily_weather_df = helpers.get_weather_data(start_date, end_date)

In [ ]:
# Loop through each day and concat date, daily bixi count, all weather variables to the main dataframe.
increment_date = start_date

while(end_date >= increment_date):
  temp_daily_weather_df = daily_weather_df.loc[pd.to_datetime(daily_weather_df['DATE']).dt.date == increment_date]
  daily_count = helpers.get_daily_count(increment_date)['f0_'][0] 
  temp_df =  pd.DataFrame(data = {
  'DATE': [increment_date], 
  'DAILY_COUNT': [daily_count], 
  'MEAN_TEMP': [temp_daily_weather_df['MEAN_TEMP'].iloc[0]],
  'TOTAL_RAIN': [temp_daily_weather_df['TOTAL_RAIN'].iloc[0]],
  'TOTAL_SNOW': [temp_daily_weather_df['TOTAL_SNOW'].iloc[0]],
  })

  main_df = pd.concat([main_df, temp_df])
  print(main_df.tail(1))
  increment_date += relativedelta(days=1)

print(main_df.shape)

In [124]:
main_df['DATE'].tail()

0    2022-05-27
0    2022-05-28
0    2022-05-29
0    2022-05-30
0    2022-05-31
Name: DATE, dtype: object